In [1]:
import spotiphy
import numpy as np
import pandas as pd
import matplotlib as mpl
import scanpy as sc
import squidpy as sq
import importlib as imp

In [39]:
sample = '8'
results_folder = 'Results/Breast cancer/Simu ' + sample + '/'
adata_sc = sc.read_h5ad("F:/Ziqian Zheng/Spotiphy_data/BreastCancerData/bcsc_down.h5ad")
adata_st = sc.read_h5ad("F:/Ziqian Zheng/Spotiphy_data/BreastCancerData/Simulated ST/ST_Simulated_BC"+sample+".h5ad")
key_type = 'broad_cell_type'
type_list = sorted(list(adata_sc.obs[key_type].unique()))
print(f'There are {len(type_list)} cell types: {type_list}')

There are 10 cell types: ['basal', 'bcells', 'fibroblasts', 'lumhr', 'lumsec', 'lymphatic', 'myeloid', 'pericytes', 'tcells', 'vascular']


In [40]:
adata_sc_orig = adata_sc.copy()
adata_st_orig = adata_st.copy()

In [41]:
adata_sc.obs.value_counts(key_type)

broad_cell_type
tcells         43896
myeloid        22877
vascular       15000
bcells         12510
pericytes      10000
lumsec          6401
basal           5000
fibroblasts     5000
lymphatic       5000
lumhr           3599
Name: count, dtype: int64

### Deconvolution

In [42]:
adata_sc, adata_st = spotiphy.sc_reference.initialization(adata_sc, adata_st, verbose=1)

Convert expression matrix to array: 4.58s
Normalization: 42.84s
Filtering: 48.1s
Find common genes: 0.15s


In [43]:
marker_gene_dict = spotiphy.sc_reference.marker_selection_1(adata_sc, key_type=key_type, return_dict=True, n_select=50,
                                                            threshold_p=0.1, threshold_fold=1.5, q=0.15)
marker_gene = []
marker_gene_label = []
for type_ in type_list:
    marker_gene.extend(marker_gene_dict[type_])
    marker_gene_label.extend([type_]*len(marker_gene_dict[type_]))
marker_gene_df = pd.DataFrame({'gene':marker_gene, 'label':marker_gene_label})
marker_gene_df.to_csv(results_folder+'marker_gene.csv')
adata_sc = adata_sc[:, marker_gene]
adata_st = adata_st[:, marker_gene]

In [44]:
sc_ref = spotiphy.construct_sc_ref(adata_sc, key_type=key_type)
spotiphy.sc_reference.plot_heatmap(adata_sc, key_type, save=True, out_dir=results_folder)

10it [00:00, 52.47it/s]
sc_reference.py (354): Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


In [45]:
X = np.array(adata_st.X)
pyro_params = spotiphy.deconvolution.deconvolute(X, sc_ref, n_epoch=7000, plot=True, batch_prior=2)
sigma = pyro_params['sigma'].cpu().detach().numpy()
mean_exp = np.array([np.mean(np.sum(adata_sc.X[adata_sc.obs[key_type]==type_list[i]], axis=1)) for i in range(len(type_list))])
cell_proportion = sigma/mean_exp
cell_proportion = cell_proportion/np.sum(cell_proportion, axis=1)[:, np.newaxis]
adata_st.obs[type_list] = cell_proportion
np.save(results_folder+'proportion.npy', cell_proportion)

100%|██████████| 7000/7000 [03:40<00:00, 31.78it/s]
deconvolution.py (103): Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
2037772055.py (7): Trying to modify attribute `.obs` of view, initializing view as actual.


In [46]:
cell_proportion = np.load(results_folder+'proportion.npy')
adata_st.obs[type_list] = cell_proportion
type_list_order = type_list

In [47]:
vmax = np.quantile(adata_st.obs[type_list_order].values, 0.98, axis=0)
vmax[vmax<0.05] = 0.05
with mpl.rc_context({'figure.figsize': [4, 4.5], 'figure.dpi': 600}):
    ax = sc.pl.spatial(adata_st, cmap='magma', color=type_list_order, img_key=None, vmin=0, vmax=list(vmax), ncols=4,
                       size=0.38, show=False, )
    ax[0].get_figure().savefig(results_folder+'spotiphy.jpg', bbox_inches='tight')

In [48]:
%matplotlib agg
%matplotlib agg
vmax = np.quantile(adata_st.obs[type_list_order].values, 0.98, axis=0)
vmax[vmax<0.05] = 0.05
with mpl.rc_context({'figure.figsize': [5, 5], 'figure.dpi': 800}):
    for i, cell_type in enumerate(type_list):
        ax = sc.pl.spatial(adata_st, cmap='magma', color=cell_type, img_key=None, vmin=0, vmax=vmax[i], size=0.38, show=False)
        cell_type = "".join(x for x in cell_type if x.isalnum())
        ax[0].get_figure().savefig(results_folder+'individual/'+cell_type+'.jpg')

### Decomposition

In [49]:
cell_proportion = np.load(results_folder+'proportion.npy')
n_cell = np.repeat(10, len(adata_st))
adata_st = adata_st_orig.copy()
adata_sc = adata_sc_orig.copy()
adata_st_decomposed = spotiphy.deconvolution.decomposition(adata_st, adata_sc, key_type, cell_proportion, save=True,
                                                           out_dir=results_folder, threshold=0.1, verbose=1, filtering_gene=True,  
                                                           spot_location=adata_st.obsm['X_spatial'], n_cell=n_cell, 
                                                           filename='ST_decomposition_simu_BC'+sample+'.h5ad')

Prepared proportion data. Time use 0.12
Initialized scRNA and ST data. Time use 98.54


10it [00:11,  1.11s/it]


Processed scRNA and ST data. Time use 86.02
Decomposition complete. Time use 10.02
Constructed ST decomposition data file. Time use 7.32
Saved file to output folder. Time use 3.96


In [50]:
adata_st_decomposed.obs

,cell_type,spot_name,n_cell,location_x,location_y
0,basal,AAACATGGTGAGAGGA-1_10,2,235.414641,-1504.043978
1,basal,AAACATGGTGAGAGGA-1_10,2,235.414641,-1504.043978
2,basal,AAACCGTTCGTCCAGG-1_10,4,755.418948,-1287.609170
3,basal,AAACCGTTCGTCCAGG-1_10,4,755.418948,-1287.609170
4,basal,AAACCGTTCGTCCAGG-1_10,4,755.418948,-1287.609170
...,...,...,...,...,...
28005,vascular,TTGGGACACTGCCCGC-1_10,1,1101.477392,-769.599903
28006,vascular,TTGGTCACACTCGTAA-1_10,1,829.612843,-1201.229359
28007,vascular,TTGTCACCGCGGTATC-1_10,1,297.045181,-1309.932043
28008,vascular,TTGTGATCTGTTCAGT-1_10,1,1610.967302,-1652.647448
